In [1]:
from haystack_experimental.chat_message_stores.in_memory import InMemoryChatMessageStore
from haystack_experimental.components.retrievers import ChatMessageRetriever
from haystack_experimental.components.writers import ChatMessageWriter
from haystack.dataclasses import ChatMessage
from haystack.components.joiners import ListJoiner
from haystack import Pipeline, component
from typing import List
from haystack.components.builders import ChatPromptBuilder
import os
import json
import mlflow
import requests


In [2]:
#load model tfidf vectorizer
mlflow.set_tracking_uri('sqlite:///mlflow.db')
tfidf_vectorizer = mlflow.sklearn.load_model("models:/tfidf_vectorizer/1")

2025/12/02 08:58:31 INFO mlflow.store.db.utils: Creating initial MLflow database tables...
2025/12/02 08:58:31 INFO mlflow.store.db.utils: Updating database tables
2025-12-02 08:58:31 INFO  [alembic.runtime.migration] Context impl SQLiteImpl.
2025-12-02 08:58:31 INFO  [alembic.runtime.migration] Will assume non-transactional DDL.
2025-12-02 08:58:31 INFO  [alembic.runtime.migration] Context impl SQLiteImpl.
2025-12-02 08:58:31 INFO  [alembic.runtime.migration] Will assume non-transactional DDL.
2025/12/02 08:58:31 INFO mlflow.store.db.utils: Creating initial MLflow database tables...
2025/12/02 08:58:31 INFO mlflow.store.db.utils: Updating database tables
2025-12-02 08:58:31 INFO  [alembic.runtime.migration] Context impl SQLiteImpl.
2025-12-02 08:58:31 INFO  [alembic.runtime.migration] Will assume non-transactional DDL.


In [3]:
memory_store = InMemoryChatMessageStore()
memory_retriever = ChatMessageRetriever(memory_store)
memory_writer = ChatMessageWriter(memory_store)

In [4]:
system_message = ChatMessage.from_system("You are a helpful assistant that answers questions based on the provided context.")
user_message_template = """
Answer the question based on the user query, please pay attention to the chat history:
chat_history:
{% for memory in memories %}
    {{memory.text}}
{% endfor %}

query:{{query}}
answer:
"""
user_message = ChatMessage.from_user(user_message_template)

In [ ]:
# llm model
@component
class GroqLLM:
    def __init__(self, model_name="meta-llama/llama-4-maverick-17b-128e-instruct", api_key=None):
        self.api_key = api_key
        self.model_name = model_name
    
    #list of chat message
    @component.output_types(output=List[ChatMessage])
    def run(self, prompt: List[ChatMessage]):
        user_prompt ="".join([msg.text for msg in prompt])
        
        url = "https://api.groq.com/openai/v1/chat/completions"
        headers = {
            "Authorization": f"Bearer {self.api_key}",
            "Content-Type": "application/json"
        }

        payload = {
            "model": self.model_name,
            "messages": [{"role": "user", "content": user_prompt}],
            "temperature": 0.7,
            "max_tokens": 300
        }

        response = requests.post(url, headers=headers, json=payload)
        try:
            data = response.json()
        except Exception:
            raise ValueError("Gagal parse JSON dari Groq API: ", response.text)

        # Debug untuk melihat isi JSON asli
        if "choices" not in data:
            raise ValueError(
                "Groq API tidak mengembalikan 'choices'.\n"
                f"Status Code: {response.status_code}\n"
                f"Response JSON:\n{json.dumps(data, indent=2)}"
            )

        # Jika OK, ambil isi respon
        result = data["choices"][0]["message"]["content"]
        return {"output": [ChatMessage.from_assistant(result)]}
    

In [6]:
@component
class PredictionCategory:
    def __init__(self,model_name :str , version : int):
        self.model_uri = f"models:/{model_name}_model/{version}"
        self.model = mlflow.sklearn.load_model(self.model_uri)
    
    @component.output_types(category=str)
    def run(self, input_data):
        transform_tfidf = tfidf_vectorizer.transform([input_data])
        category = self.model.predict(transform_tfidf)
        return {"category" : category[0]}

In [7]:
groq_llm = GroqLLM(api_key=os.getenv("GROQ_API_KEY"))
pipeline = Pipeline()
pipeline.add_component("prompt_builder", ChatPromptBuilder(variables=["query","memories"], required_variables=["query","memories"]))
pipeline.add_component("generator",groq_llm)
pipeline.add_component("joiner",ListJoiner(List[ChatMessage]))
pipeline.add_component("memory_retriever", memory_retriever)
pipeline.add_component("memory_writer", memory_writer)


pipeline.connect("prompt_builder.prompt", "generator.prompt")
pipeline.connect("generator.output", "joiner")
pipeline.connect("joiner", "memory_writer")
pipeline.connect("memory_retriever", "prompt_builder.memories")


🚅 Components
  - prompt_builder: ChatPromptBuilder
  - generator: GroqLLM
  - joiner: ListJoiner
  - memory_retriever: ChatMessageRetriever
  - memory_writer: ChatMessageWriter
🛤️ Connections
  - prompt_builder.prompt -> generator.prompt (list[ChatMessage])
  - generator.output -> joiner.values (List[ChatMessage])
  - joiner.values -> memory_writer.messages (List[ChatMessage])
  - memory_retriever.messages -> prompt_builder.memories (List[ChatMessage])

In [11]:
while True:
    messages = [system_message, user_message]
    query = input("Please input your question or type 'exit' to quit.\n")
    if query.lower() == "exit":
        break
    res = pipeline.run(
        data={
            "prompt_builder": {
                "query": query,
                "template":messages
            },
            "joiner":{
                "values": [ChatMessage.from_user(query)]
            }
        },
        include_outputs_from=["generator"]
    )
    response_text = res['generator']['output'][0]._content[0].text
    print(response_text)

Halo! Ada yang bisa saya bantu?
Baik! Ada yang bisa saya bantu?
